# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='https://i.imgur.com/6n64KAw.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

## Install PyTorch (setup for Google Colab)

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

### Mount your Google Drive


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download the dataset (code setup for Google Colab)

In [0]:
#!wget "https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip" -P "drive/My Drive/PyTorch Challenge/"
#!unzip "drive/My Drive/PyTorch Challenge/flower_data.zip" -d "drive/My Drive/PyTorch Challenge/Side Project/Flower_dataset/"
#!wget "https://github.com/udacity/pytorch_challenge/blob/master/cat_to_name.json" -P "drive/My Drive/PyTorch Challenge/cat_to_name.json"

## Import the test environment 
### The functions below were used for calculating the accuracy with a function from built by the community

In [0]:
!git clone https://github.com/GabrielePicco/deep-learning-flower-identifier
!pip install requests
!pip install airtable
import sys
sys.path.insert(0, 'deep-learning-flower-identifier')
from test_model_pytorch_facebook_challenge import publish_evaluated_model, calc_accuracy

fatal: destination path 'deep-learning-flower-identifier' already exists and is not an empty directory.


## Import (for Google Colab)

In [0]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

Uninstalling Pillow-5.4.1:
  Successfully uninstalled Pillow-5.4.1
    100% |████████████████████████████████| 2.0MB 14.4MB/s 
4.0.0


In [0]:
#!pip install --no-cache-dir -I pillow
# Imports here
%matplotlib inline
import time
import os
import json
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
from google.colab import files
torch.manual_seed(42)

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [0]:
data_dir = 'drive/My Drive/PyTorch Challenge/Flower_dataset/flower_data/'
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')

dirs = {'train': train_dir, 
        'valid': valid_dir}

In [0]:
size = 224
data_transforms = data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(25),
        transforms.RandomResizedCrop(size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size + 32),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
} 

image_datasets = {x: datasets.ImageFolder(dirs[x],   transform=data_transforms[x]) for x in ['train', 'valid']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) 
                              for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [0]:
with open('drive/My Drive/PyTorch Challenge/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [0]:
#Load pretrained model    
model = models.densenet161(pretrained=True)

#freeze parameters so that we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [0]:
#Uncomment if you want to print model architecture
#model

In [0]:
# Adjust number of classes
model.classifier = nn.Linear(2208, 102)

In [0]:
def train_model(model, criteria, optimizer, scheduler, num_epochs=25, device='cuda'):
    model.to(device)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criteria(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
# Criteria NLLLoss which is recommended with Softmax final layer
criteria = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = optim.SGD(model.classifier.parameters(), lr=0.006, momentum=0.9, nesterov=True)
# Decay LR by a factor of 0.1 every 4 epochs
sched = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# Number of epochs
eps=14

In [0]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ft = train_model(model, criteria, optimizer, sched, eps, device)

Epoch 0/13
----------
train Loss: 2.1059 Acc: 0.5726
valid Loss: 0.6228 Acc: 0.8741

Epoch 1/13
----------
train Loss: 0.7747 Acc: 0.8410
valid Loss: 0.3330 Acc: 0.9328

Epoch 2/13
----------
train Loss: 0.5531 Acc: 0.8863
valid Loss: 0.2666 Acc: 0.9413

Epoch 3/13
----------
train Loss: 0.4573 Acc: 0.9009
valid Loss: 0.2428 Acc: 0.9425

Epoch 4/13
----------
train Loss: 0.4037 Acc: 0.9144
valid Loss: 0.2097 Acc: 0.9511

Epoch 5/13
----------
train Loss: 0.3633 Acc: 0.9171
valid Loss: 0.1910 Acc: 0.9548

Epoch 6/13
----------
train Loss: 0.3492 Acc: 0.9176
valid Loss: 0.1984 Acc: 0.9548

Epoch 7/13
----------
train Loss: 0.2864 Acc: 0.9350
valid Loss: 0.1813 Acc: 0.9560

Epoch 8/13
----------
train Loss: 0.2899 Acc: 0.9357
valid Loss: 0.1691 Acc: 0.9584

Epoch 9/13
----------
train Loss: 0.2823 Acc: 0.9396
valid Loss: 0.1646 Acc: 0.9621

Epoch 10/13
----------
train Loss: 0.2810 Acc: 0.9394
valid Loss: 0.1745 Acc: 0.9560

Epoch 11/13
----------
train Loss: 0.2771 Acc: 0.9417
valid Loss

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [0]:
# Find total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

26,697,318 total parameters.
225,318 training parameters.


In [0]:
model_file_name = 'classifier_densenet161.pth'
path = F"drive/My Drive/PyTorch Challenge/{model_file_name}" 
model.class_to_idx = image_datasets['train'].class_to_idx
model.cpu()
torch.save({'arch': 'densenet161',
            'state_dict': model.state_dict(), 
            'class_to_idx': model.class_to_idx,
            'optimizer_state_dict': optimizer.state_dict,
            'criterion':criteria},
             path)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [0]:
def load_model(checkpoint_path):
    chpt = torch.load(checkpoint_path)
    pretrained_model = getattr(models, chpt['arch'])
    if callable(pretrained_model):
        model = pretrained_model(pretrained=True)
        for param in model.parameters():
            param.requires_grad = False
    else:
        print("Sorry base architecture not recognized")
    
    model.class_to_idx = chpt['class_to_idx']
    
    # Create the classifier
    model.classifier = nn.Linear(2208, 102)
    
    model.load_state_dict(chpt['state_dict'])
    
    return model

In [0]:
model = load_model('drive/My Drive/PyTorch Challenge/classifier_densenet161.pth')
calc_accuracy(model, input_image_size=224, testset_path=valid_dir)

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.90625
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.875
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.944444477558136
Mean accuracy: 0.9642094373703003


0.96420944

### Retrain only specific layers

In [0]:
# This is for DenseNet architecture, you need to adjust the method for each architecture
for name in model.children():
  for child, config in name.named_children():
    if child in ['denseblock4', 'norm5']:
      print(str(child) + ' is unfrozen')
      for param in config.parameters():
          param.requires_grad = True
    else:
      print(str(child) + ' is frozen')
      for param in config.parameters():
          param.requires_grad = False

conv0 is frozen
norm0 is frozen
relu0 is frozen
pool0 is frozen
denseblock1 is frozen
transition1 is frozen
denseblock2 is frozen
transition2 is frozen
denseblock3 is frozen
transition3 is frozen
denseblock4 is unfrozen
norm5 is unfrozen


In [0]:
#settings to train using different optimizer
criteria = nn.CrossEntropyLoss()
#adjust optimizer 
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.00006, momentum=0.9, nesterov=True)
# Use scheduler
sched = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# Number of epochs
eps=12

In [0]:
#train model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_full = train_model(model, criteria, optimizer, sched, eps, device)

Epoch 0/11
----------
train Loss: 0.1542 Acc: 0.9644
valid Loss: 0.0872 Acc: 0.9817

Epoch 1/11
----------
train Loss: 0.1539 Acc: 0.9621
valid Loss: 0.0892 Acc: 0.9768

Epoch 2/11
----------
train Loss: 0.1507 Acc: 0.9669
valid Loss: 0.0864 Acc: 0.9804

Epoch 3/11
----------
train Loss: 0.1377 Acc: 0.9687
valid Loss: 0.0889 Acc: 0.9817

Epoch 4/11
----------
train Loss: 0.1537 Acc: 0.9640
valid Loss: 0.0914 Acc: 0.9804

Epoch 5/11
----------
train Loss: 0.1379 Acc: 0.9681
valid Loss: 0.0864 Acc: 0.9804

Epoch 6/11
----------
train Loss: 0.1671 Acc: 0.9615
valid Loss: 0.0899 Acc: 0.9792

Epoch 7/11
----------
train Loss: 0.1487 Acc: 0.9643
valid Loss: 0.0874 Acc: 0.9817

Epoch 8/11
----------
train Loss: 0.1529 Acc: 0.9663
valid Loss: 0.0872 Acc: 0.9817

Epoch 9/11
----------
train Loss: 0.1488 Acc: 0.9663
valid Loss: 0.0890 Acc: 0.9792

Epoch 10/11
----------
train Loss: 0.1483 Acc: 0.9672
valid Loss: 0.0893 Acc: 0.9804

Epoch 11/11
----------
train Loss: 0.1570 Acc: 0.9644
valid Loss

In [0]:
#save model
model_file_name = 'classifier_densenet161_2.pth'
path = F"drive/My Drive/PyTorch Challenge/{model_file_name}" 
model.class_to_idx = image_datasets['train'].class_to_idx
model.cpu()
torch.save({'arch': 'densenet161',
            'state_dict': model.state_dict(), 
            'class_to_idx': model.class_to_idx,
            'optimizer_state_dict': optimizer.state_dict,
            'criterion':criteria},
             path)

In [0]:
#calculate accuracy
calc_accuracy(model, input_image_size=224, testset_path=valid_dir)

Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Mean accuracy: 0.9819711446762085


0.98197114

## Publish the result on the Airtable shared leaderboard

In [0]:
#publish_evaluated_model(model, input_image_size=224,  username="@Slack.Username", model_name="VGG19", optim="Adam", criteria="NLLLoss", scheduler="StepLR", epoch=5)